# Traffic in a NY's district

In [1]:
import numpy as np 
import networkx as nx 
from graph import Graph,rounding, latex

In [2]:
W = np.array([[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              ])

In [3]:
G = Graph(W)
#G.draw_graph()

In [4]:
import time
RANDOM_SEED = int((time.time()%10)*1000)
print("Seed = %d" % RANDOM_SEED)
np.random.seed(RANDOM_SEED)

Seed = 2075


In [5]:
nodes = W.shape[0]

m = 17

# link length computed generate a random value
seg_length = 2000*np.random.rand(m,1)

# flow value of each link computed generating a random value
f = 600 + 100*(2*np.random.rand(m,1) - 1)

# Capacity of each link computed generating random value
C = 1000 + 100*(2*np.random.rand(m,1) - 1)

# assuming speed of 14 m/s
speed = 14

# minimum travel time for each segment
t_e = seg_length/speed

In [6]:
edges = []

#looking at the adjacency matrix we determine al the link
for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        if W[i,j] != 0:
            edges.append((i,j))
n_edges = len(edges)

### Incidence matrix

In [7]:
B = np.zeros((nodes,n_edges))

# based on the link found we compute node-link incidence matrix B
for index,edge in enumerate(edges):
    node_i, node_j = edge
    B[node_i,index] = 1
    B[node_j,index] = -1
    
display(latex(B))   

Matrix([
[ 1.0,  1.0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[-1.0,    0,  1.0,  1.0,  1.0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,    0, -1.0,    0,    0,  1.0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,    0,    0, -1.0,    0,    0,  1.0,    0,    0,    0, -1.0,    0,    0,    0,    0,    0,    0],
[   0,    0,    0,    0, -1.0,    0,    0,  1.0,    0,    0,    0,    0, -1.0,    0, -1.0,    0,    0],
[   0,    0,    0,    0,    0, -1.0,    0, -1.0,  1.0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,    0,    0,    0,    0,    0,    0,    0, -1.0,  1.0,    0,    0,    0,    0,    0,    0,    0],
[   0, -1.0,    0,    0,    0,    0,    0,    0,    0,    0,  1.0,  1.0,    0,    0,    0,    0,    0],
[   0,    0,    0,    0,    0,    0, -1.0,    0,    0,    0,    0,    0,  1.0,  1.0,    0,    0,    0],
[   0,    0,    0,    0,    0,    0,    0,    0,    0, 

In [20]:
B.shape

(12, 17)

___

### Shortest path

In [8]:
W_times = np.zeros((nodes, n_edges))

# To compute the shortest time we set a new adjacency matrix with time travel values
for indx, edge in enumerate(edges):
    i,j = edge
    W_times[i,j] = t_e[indx]

In [9]:
graph = Graph(W_times)

#find the shortest path step fro 0 to 11 (we must consider that indices in python start from 0 so they correspond to a +1 indices in the exercise)
shortest_path = graph.dist(0,11)[1]

# display shortest path
display(shortest_path)

time_shortest_path = 0

# compute total time for shortest path
for node in shortest_path:
    time_shortest_path += t_e[node]

time_shortest_path = time_shortest_path[0]
print('The minimum time to go from 1 to 12 in an empty network is: ', round(time_shortest_path,3), 's')

[7, 10, 11]

The minimum time to go from 1 to 12 in an empty network is:  258.902 s


___

### Maximum flow

In [10]:
graph = nx.DiGraph()

# creating a new adjacency matrix in order to compute max flow
for indx, capacity in enumerate(C):
    i,j = edges[indx]
    cap = capacity[0]
    graph.add_edge(str(i),str(j), capacity = cap)

tau_max = nx.maximum_flow_value(graph,"0","11")
tau_max = round(tau_max,3)
print('The maximum flow is:', tau_max)

The maximum flow is: 1821.078


___

### External inflows/outflows

In [11]:
nu = B@f
nu.reshape((1,len(nu)))[0]
external_inflow = nu.copy()
external_outflow = nu.copy()

for i in range(len(external_inflow)):
    external_inflow[i] = max(external_inflow[i],0)
print("External inflow value:")
display(latex(external_inflow).T)

for i in range(len(external_outflow)):
    external_outflow[i] = max(-external_outflow[i],0)
print("External ouflow value:")
display(latex(external_outflow).T)
    
print("Net exogenous flow:")
display(latex(nu).T)


External inflow value:


Matrix([[1115.80171130084, 1207.22398680057, 137.309464850355, 0, 0, 0, 123.227038781501, 521.676307870786, 540.24576558741, 573.395298216262, 0, 0]])

External ouflow value:


Matrix([[0, 0, 0, 725.692682330812, 1152.78805316367, 705.761424499189, 0, 0, 0, 0, 500.22499917926, 1134.4124142348]])

Net exogenous flow:


Matrix([[1115.80171130084, 1207.22398680057, 137.309464850355, -725.692682330812, -1152.78805316367, -705.761424499189, 123.227038781501, 521.676307870786, 540.24576558741, 573.395298216262, -500.22499917926, -1134.4124142348]])

___

### Social Optimum

In [12]:
import cvxpy as cvx

nu[1:11] = 0
nu[-1] = -nu[0]

# varaible of the optimization problem
fe = cvx.Variable((n_edges,1))

# cost function
object = cvx.Minimize(cvx.sum(cvx.multiply(cvx.multiply(t_e,C),cvx.inv_pos(1 - cvx.multiply(fe,1/C))) - cvx.multiply(t_e,C)))

# constraints
constraints = [B@fe == nu, 0 <= fe, fe <= C]

problem = cvx.Problem(object, constraints)

result = problem.solve()

fe_social_optimum = fe.value
social_optimum_cost = problem.value

print("Social optimum cost:", round(social_optimum_cost,1))

print("Social optimum flow values:")
display(latex(fe_social_optimum))

Social optimum cost: 729017.1
Social optimum flow values:


Matrix([
[   481.226768122911],
[   634.574943179578],
[   183.283148373975],
[   45.4016040726926],
[   252.542015676939],
[   183.283148374582],
[     155.2358141377],
[    252.54201782269],
[   435.825166197673],
[   435.825166196582],
[   109.834210065099],
[   524.740733114156],
[1.64231896392642e-6],
[   155.235812494684],
[ 5.0248021126977e-7],
[   435.825165693216],
[    679.97654560756]])

___

### Wardrop equilibrium

In [13]:
fe_wardrop = cvx.Variable((n_edges,1))

object = cvx.Minimize( cvx.sum( cvx.multiply( cvx.multiply(-t_e,C), cvx.log(1- cvx.multiply(fe_wardrop,1/C)) )))

constraints = [B@fe_wardrop == nu, 0 <= fe_wardrop, fe_wardrop <= C]

problem = cvx.Problem(object, constraints)

result = problem.solve()

print("Wardrop flow values:")
fe_wardrop= fe_wardrop.value
display(latex(fe_wardrop).T)

Wardrop flow values:


Matrix([[393.095092359098, 722.706611055796, 96.3060421025673, 40.5577162811076, 256.231333975418, 96.3060421025642, 55.1778783522035, 256.231335885811, 352.537377988374, 352.537377988374, 14.6201620710897, 708.08644898468, 1.30883077606443e-6, 55.177877043386, 6.01561011435913e-7, 352.537377386811, 763.264326028089]])

___

### Price of anarchy

In [14]:
cost_at_wardrop = 0

for i in range(len(t_e)):
    cost_at_wardrop += (fe_wardrop[i]*t_e[i])/(1 - (fe_wardrop[i]/C[i]) )

print('Cost at Wardrop:', rounding(cost_at_wardrop,3)[0])

# Price of Anarchy
PoA = cost_at_wardrop/social_optimum_cost 
print('The Price of Anarchy is:',round(PoA[0],3))

Cost at Wardrop: 842727.955
The Price of Anarchy is: 1.156


___

### Wardrop equilibrium with tools

In [15]:
tolls = []

#defining the value of the tolls
for i in range(len(fe_social_optimum)):
    d_de =  C[i]*t_e[i]/(C[i] - fe_social_optimum[i])**2
    tolls.append(fe_social_optimum[i]*d_de)

tolls = np.asarray(tolls)

In [16]:
fe_tolls = cvx.Variable((n_edges,1))

object = cvx.Minimize( cvx.sum( cvx.multiply( cvx.multiply(-t_e,C), cvx.log(1- cvx.multiply(fe_tolls,1/C)) ) + cvx.multiply(tolls,fe_tolls)))

constraints = [B@fe_tolls == nu, 0 <= fe_tolls, fe_tolls <= C]

problem = cvx.Problem(object, constraints)

result = problem.solve()

fe_tolls = fe_tolls.value

cost_at_wardrop_tolls = 0

for i in range(len(t_e)):
    cost_at_wardrop_tolls += (fe_tolls[i]*t_e[i])/(1 - (fe_tolls[i]/C[i]) )
    

print('Cost at Wardrop with tolls:', rounding(cost_at_wardrop,3)[0])
PoA = cost_at_wardrop_tolls/social_optimum_cost 
print('The Price of Anarchy with tolls is:',round(PoA[0],3))

Cost at Wardrop with tolls: 842727.955
The Price of Anarchy with tolls is: 1.0


___

Let now consider a new cost:
$$c_e(f_e) = f_e(d_e(f_e) - t_e)$$


We want tocompute the new social optimum and the we want to construct the tolls that mak the Wardrop equilibrium coincides with the social optimum.

In [17]:
fe_new = cvx.Variable((n_edges,1))

object = cvx.Minimize(cvx.sum(cvx.multiply(cvx.multiply(t_e,C),cvx.inv_pos(1 - cvx.multiply(fe_new,1/C))) \
                              - cvx.multiply(t_e,C) - cvx.multiply(fe_new,t_e)))

constraints = [B@fe_new == nu, 0 <= fe_new, fe_new <= C]

problem = cvx.Problem(object, constraints)

result = problem.solve()

fe_new_social_optimum = fe_new.value
new_social_optimum_cost = problem.value

print("Social optimum flows:")
display(latex(fe_new_social_optimum))

print("Social optimum cost:",round(new_social_optimum_cost,2) )

Social optimum flows:


Matrix([
[    502.351000330849],
[    613.450710976905],
[    214.898628640289],
[    51.5249225722751],
[    235.927449117274],
[    214.898628639689],
[    235.288461906153],
[    235.927585035937],
[    450.826213675354],
[    450.826213677349],
[    183.763539335963],
[    429.687171639614],
[0.000133763514453369],
[    235.288328139211],
[ 2.15536272801287e-6],
[    450.826211524266],
[    664.975499773314]])

Social optimum cost: 340087.91


In [18]:
tolls_new = []

for i in range(len(fe_new_social_optimum)):
    toll =  (fe_new_social_optimum[i]*C[i]*t_e[i])/((C[i] - fe_new_social_optimum[i])**2 ) 
    tolls_new.append(toll)

tolls_new = np.asarray(tolls_new) 

In [19]:
fe_tolls_new = cvx.Variable((n_edges,1))

object = cvx.Minimize( cvx.sum( cvx.multiply( cvx.multiply(-t_e,C), cvx.log(1- cvx.multiply(fe_tolls_new,1/C)) )+ cvx.multiply(tolls_new,fe_tolls_new)- cvx.multiply(fe_tolls_new,t_e)))

constraints = [B@fe_tolls_new == nu, 0 <= fe_tolls_new, fe_tolls_new <= C]

problem = cvx.Problem(object, constraints)

result = problem.solve()

fe_tolls_new = fe_tolls_new.value
display(latex(fe_tolls_new).T)
display(latex(fe_new_social_optimum).T)

cost_at_wardrop_tolls_new = 0

for i in range(len(t_e)):
    cost_at_wardrop_tolls_new += (fe_tolls_new[i]*t_e[i])/(1 - (fe_tolls_new[i]/C[i]) ) - fe_tolls_new[i]*t_e[i]

print('Cost at Wardrop with tolls:', rounding(cost_at_wardrop_tolls_new,2)[0])
PoA = cost_at_wardrop_tolls_new/new_social_optimum_cost 

print('The Price of Anarchy with tolls is:',round(PoA[0],3))

Matrix([[502.332401022851, 613.469300722273, 214.912828142045, 51.4760472104401, 235.943525670356, 214.912828142048, 235.24112620707, 235.943556983356, 450.856385125419, 450.856385125441, 183.76507899664, 429.704221725615, 3.07097762758036e-5, 235.241095497263, 6.03209157061984e-7, 450.856384522254, 664.945317222846]])

Matrix([[502.351000330849, 613.450710976905, 214.898628640289, 51.5249225722751, 235.927449117274, 214.898628639689, 235.288461906153, 235.927585035937, 450.826213675354, 450.826213677349, 183.763539335963, 429.687171639614, 0.000133763514453369, 235.288328139211, 2.15536272801287e-6, 450.826211524266, 664.975499773314]])

Cost at Wardrop with tolls: 340087.9
The Price of Anarchy with tolls is: 1.0
